In [12]:
import polars as pl
import json
import os
import shutil
import numpy as np

In [13]:
# Load furniture categories
source_folder = "3D-FUTURE-scene"
threedfuture_file_path = source_folder + '/train/train_set.json'

with open(threedfuture_file_path, 'r') as json_file:
    data = json.load(json_file)

for key in data:
    print(key, len(data[key]))

categories 34
images 14761
annotations 73513


In [14]:
# Create Dataframe tables
# Categories
categories_data = data['categories']
df_categories = pl.DataFrame(categories_data)

df_categories.head()

id,super-category,category
i64,str,str
1,"""Cabinet/Shelf/…","""Children Cabin…"
2,"""Cabinet/Shelf/…","""Nightstand"""
3,"""Cabinet/Shelf/…","""Bookcase / jew…"
4,"""Cabinet/Shelf/…","""Wardrobe"""
5,"""Cabinet/Shelf/…","""Coffee Table"""


In [15]:
# Images
images_data = data['images']
df_images = pl.DataFrame(images_data)

df_images.head()

id,width,height,file_name
i64,i64,i64,str
1,1200,1200,"""0000000"""
2,1200,1200,"""0000001"""
3,1200,1200,"""0000002"""
4,1200,1200,"""0000003"""
5,1200,1200,"""0000004"""


In [16]:
source_folder = "./"
file_path = source_folder + 'blip_anotation_3D_DS.csv'

In [17]:
df = pl.read_csv(file_path)

df.head()

,annotation_id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion,width,height,file_name,caption,Room
i64,i64,i64,i64,i64,str,str,str,f64,str,str,str,str,str,i64,i64,i64,str,str
0,1,1,33,13073,"""[526.0, 56.0, …","""da64b6ef-d704-…","""da64b6ef-d704-…",0.958973,"""Modern""",null,null,"""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom"""
1,2,1,2,8014,"""[1140.0, 795.0…","""4d5ff861-d5ce-…","""4d5ff861-d5ce-…",null,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom"""
2,3,1,23,24436,"""[0.0, 603.0, 2…","""a939be24-9c3d-…","""a939be24-9c3d-…",0.958973,"""Light Luxury""","""Gold Foil""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom"""
3,4,1,4,211496,"""[173.0, 287.0,…","""1d9f21cd-359f-…","""1d9f21cd-359f-…",0.958973,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom"""
4,5,1,13,235956,"""[414.0, 553.0,…","""433123cb-172e-…","""433123cb-172e-…",0.958973,"""Mediterranean""","""Striped Grid""","""Rough Cloth""","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom"""


In [18]:
df_joined = df.join(df_categories, left_on="category_id", right_on="id")
df_joined.head()

,annotation_id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion,width,height,file_name,caption,Room,super-category,category
i64,i64,i64,i64,i64,str,str,str,f64,str,str,str,str,str,i64,i64,i64,str,str,str,str
0,1,1,33,13073,"""[526.0, 56.0, …","""da64b6ef-d704-…","""da64b6ef-d704-…",0.958973,"""Modern""",null,null,"""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Lighting""","""Pendant Lamp"""
1,2,1,2,8014,"""[1140.0, 795.0…","""4d5ff861-d5ce-…","""4d5ff861-d5ce-…",null,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Nightstand"""
2,3,1,23,24436,"""[0.0, 603.0, 2…","""a939be24-9c3d-…","""a939be24-9c3d-…",0.958973,"""Light Luxury""","""Gold Foil""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Table""","""Dressing Table…"
3,4,1,4,211496,"""[173.0, 287.0,…","""1d9f21cd-359f-…","""1d9f21cd-359f-…",0.958973,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Wardrobe"""
4,5,1,13,235956,"""[414.0, 553.0,…","""433123cb-172e-…","""433123cb-172e-…",0.958973,"""Mediterranean""","""Striped Grid""","""Rough Cloth""","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Bed""","""King-size Bed"""


In [19]:
df_concat = df_joined.with_columns(
    pl.concat_str(
        [
            pl.col("style").fill_null(""),
            pl.col("material").fill_null("").str.to_lowercase(),
            pl.col("category").str.to_lowercase(),
        ],
        separator=" ",
    ).alias("model_description"),
)

df_concat.head()

,annotation_id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion,width,height,file_name,caption,Room,super-category,category,model_description
i64,i64,i64,i64,i64,str,str,str,f64,str,str,str,str,str,i64,i64,i64,str,str,str,str,str
0,1,1,33,13073,"""[526.0, 56.0, …","""da64b6ef-d704-…","""da64b6ef-d704-…",0.958973,"""Modern""",null,null,"""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Lighting""","""Pendant Lamp""","""Modern pendan…"
1,2,1,2,8014,"""[1140.0, 795.0…","""4d5ff861-d5ce-…","""4d5ff861-d5ce-…",null,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Nightstand""","""Korean composi…"
2,3,1,23,24436,"""[0.0, 603.0, 2…","""a939be24-9c3d-…","""a939be24-9c3d-…",0.958973,"""Light Luxury""","""Gold Foil""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Table""","""Dressing Table…","""Light Luxury c…"
3,4,1,4,211496,"""[173.0, 287.0,…","""1d9f21cd-359f-…","""1d9f21cd-359f-…",0.958973,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Wardrobe""","""Korean composi…"
4,5,1,13,235956,"""[414.0, 553.0,…","""433123cb-172e-…","""433123cb-172e-…",0.958973,"""Mediterranean""","""Striped Grid""","""Rough Cloth""","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Bed""","""King-size Bed""","""Mediterranean …"


# Concatenate room captions
For each image
BLIP descriptions + The room has {style} {category}, {style} {category}, ...
e.g. The room has Mediterranean style Coffee table, Korean style Sofa

In [21]:
df_img = df_concat.group_by(['image_id', "Room", "caption"]).agg(
    pl.col("model_description").unique().str.concat(", "),
    pl.col("model_description").unique().count().alias("item_count")
).join(
    df_images.select(pl.exclude(["width", "height"])), left_on="image_id", right_on="id"
)

# Concatenate the generated annotations with BLIP captions
df_img = df_img.with_columns(
    pl.concat_str([
        pl.col("caption").str.strip_chars(),
        pl.col("model_description").str.strip_chars(", ")
    ], separator="; ").alias("full_description")
)

# Replace file names with *.jpg format
df_img = df_img.with_columns(
    df_img["file_name"] + ".jpg",
)

print(df_img)

shape: (14_761, 7)
┌──────────┬─────────────┬───────────────┬───────────────┬────────────┬─────────────┬──────────────┐
│ image_id ┆ Room        ┆ caption       ┆ model_descrip ┆ item_count ┆ file_name   ┆ full_descrip │
│ ---      ┆ ---         ┆ ---           ┆ tion          ┆ ---        ┆ ---         ┆ tion         │
│ i64      ┆ str         ┆ str           ┆ ---           ┆ u32        ┆ str         ┆ ---          │
│          ┆             ┆               ┆ str           ┆            ┆             ┆ str          │
╞══════════╪═════════════╪═══════════════╪═══════════════╪════════════╪═════════════╪══════════════╡
│ 1        ┆ bedroom     ┆ a bedroom     ┆ Modern        ┆ 5          ┆ 0000000.jpg ┆ a bedroom    │
│          ┆             ┆ with a bed,   ┆ pendant lamp, ┆            ┆             ┆ with a bed,  │
│          ┆             ┆ dressers, …   ┆ Light Luxu…   ┆            ┆             ┆ dressers, …  │
│ 2        ┆ living room ┆ a living room ┆ Minimalist    ┆ 6          ┆ 

In [10]:
df_img.write_csv(file="3dfront-annotated.csv")

In [22]:
summary = df_img.filter(pl.col("Room").is_in([
    "living room", "bedroom", "dining room"
])).group_by(["Room", "item_count"]).count().sort(["Room", "item_count"], descending=[False, True])

summary

Room,item_count,count
str,u32,u32
"""bedroom""",8,3
"""bedroom""",7,16
"""bedroom""",6,405
"""bedroom""",5,944
"""bedroom""",4,1648
"""bedroom""",3,993
"""bedroom""",2,281
"""bedroom""",1,25
"""dining room""",8,1


In [23]:
sorted_living_df = df_img.filter(pl.col("Room") == "living room").sort(["item_count", "file_name"], descending=[True, False])
# sorted_bedroom_df = df_img.filter(pl.col("Room") == "bedroom" & pl.col("item_count") > 3).sort(["item_count", "file_name"], descending=[True, False])
# sorted_dining_df = df_img.filter(pl.col("Room") == "dining room" & pl.col("item_count") > 3).sort(["item_count", "file_name"], descending=[True, False])

In [12]:
def move_files(key, file_names):
    source_folder = "3D-FUTURE-scene"
    dest_folder = os.path.join(source_folder, "train", "image_512_" + key)

    if not os.path.exists(dest_folder):
        # If it doesn't exist, create it
        os.makedirs(dest_folder)

    for file_name in file_names:
        full_file_name = os.path.join(source_folder, "train", "image_512", file_name)
        dest = os.path.join(dest_folder, file_name)
        shutil.copy(full_file_name, dest)


In [31]:
move_files("living", sorted_living_df["file_name"])

FileNotFoundError: [Errno 2] No such file or directory: './train\\image_512\\0000072.jpg'

# Clean dataset
Images were manually sifted through and handpicked to form final dataset for training.

In [25]:
# Working with cleaned dataset
def filter_images(folder):
    main_source_folder = "3D-FUTURE-scene"

    clean_folder_path = os.path.join(main_source_folder, 'train', folder)

    # # List all filenames in the folder
    filenames = os.listdir(clean_folder_path)
    filtered_df = df_img.filter(df_img['file_name'].is_in(filenames))
    return filtered_df

In [26]:
filtered_living_df = filter_images('image_512_living_f')

In [27]:
summary_filtered_living = filtered_living_df.group_by(["Room", "item_count"]).count().sort(["Room", "item_count"], descending=[False, True])

summary_filtered_living

Room,item_count,count
str,u32,u32
"""living room""",9,1
"""living room""",8,4
"""living room""",7,11
"""living room""",6,23
"""living room""",5,67
"""living room""",4,53
"""living room""",3,21
"""living room""",1,1


In [28]:
def output_prompts(folder, df):
    main_source_folder = "3D-FUTURE-scene"
    output_directory = os.path.join(main_source_folder, 'train', folder)
    os.makedirs(output_directory, exist_ok=True)

    file_name_index = df.find_idx_by_name("file_name")
    full_description_index = df.find_idx_by_name("full_description")
    
    def output_image(row):
        name = row[file_name_index].split(".")[0]
        output_file_path = os.path.join(output_directory, f'{name}.txt')
        with open(output_file_path, "w") as file:
            file.write(row[full_description_index])
        return row

    df.map_rows(output_image)

In [29]:
output_prompts("image_512_living_f_captions", filtered_living_df)

# Prepare IKEA datasets

In [56]:
ikea_file_path = "room_scenes_blip2.csv"
df_ikea = pl.read_csv(ikea_file_path)

df_ikea.head()

,filename,caption
i64,str,str
0,"""ikea-crisp-modern-design-perfect-for-breakfast-time__1364308406824-s4""","""a kitchen with white cabinets and a table"""
1,"""ikea-kid-friendly-storage-in-traditional-style__1364308477569-s4""","""a pink room with a wooden dresser and a white dresser"""
2,"""ikea-living-room-by-day-sleeping-space-by-night.__1364309472217-s4""","""a bedroom with a bed, dresser and fireplace"""
3,"""ikea-traditional-looks-meet-modern-versatility__1364308465624-s4""","""a kitchen with white cabinets and wooden floors"""
4,"""ikea-traditional-long-lasting-furniture-for-every-generation__1364309710705-s4""","""a white table and chairs in a garden"""


In [63]:
def filter_ikea_rooms(room, df):
    df_filtered = df.filter(df['caption'].str.contains(room))
    return df_filtered

In [64]:
ikea_living_df = filter_ikea_rooms("living room", df_ikea)

In [58]:
from PIL import Image

# Function to resize an image using GPU
def resize_image(input_path, output_path, width=512, height=512):
    try:
        # Load the image using Pillow
        image = Image.open(input_path)

        resized_image = image.resize((512, 512), Image.LANCZOS)
        resized_image.save(output_path)
    except Exception as e:
        print(f"Error resizing {input_path}: {str(e)}")

In [60]:
input_directory = './ikea_room_scenes/room_scenes/'
output_directory = './ikea_room_scenes/room_scenes_512/'

ikea_filenames = os.listdir(input_directory)

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Directory '{output_directory}' has been created.")

for filename in ikea_filenames:
    resize_image(input_directory + filename, output_directory + filename)

Directory './ikea_room_scenes/room_scenes_512/' has been created.


C:\Users\JackChua\AppData\Local\Temp\ipykernel_17584\2552947371.py:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((512, 512), Image.ANTIALIAS)


In [79]:
def create_ikea_dataset(key, df):
    main_source_folder = "./ikea_room_scenes"
    input_directory = os.path.join(main_source_folder, 'room_scenes_512')
    output_directory = os.path.join(main_source_folder, 'room_scenes_512_' + key)

    output_directory_images = os.path.join(output_directory, 'images')
    output_directory_captions = os.path.join(output_directory, 'captions')

    os.makedirs(output_directory_images, exist_ok=True)
    os.makedirs(output_directory_captions, exist_ok=True)

    filename_index = df.find_idx_by_name("filename")
    caption_index = df.find_idx_by_name("caption")

    style = "; Scandinavian style"
    
    def output_image(row):
        filename = row[filename_index]
        output_file_path = os.path.join(output_directory_captions, f'{filename}.txt')
        with open(output_file_path, "w") as file:
            file.write(row[caption_index] + style)

        # Copy selected images over
        filename_jpg = filename + ".jpg"
        input_img_path = os.path.join(input_directory, filename_jpg)
        output_img_path = os.path.join(output_directory_images, filename_jpg)
        shutil.copy(input_img_path, output_img_path)
        return row

    df.map_rows(output_image)

In [80]:
create_ikea_dataset("living", ikea_living_df)

In [88]:
# Combines dataset of 
# def combine_data(key):
#     main_source_folder = "./combined_data"
#     output_directory = os.path.join(main_source_folder, key)
#     output_directory_images = os.path.join(output_directory, 'images')
#     output_directory_captions = os.path.join(output_directory, 'captions')

#     if not os.path.exists(output_directory_images):
#         os.makedirs(output_directory_images, exist_ok=True)
#     if not os.path.exists(output_directory_captions):
#         os.makedirs(output_directory_captions, exist_ok=True)

#     # 3D Future Data
#     threedfuture_main_source_folder = "./3D-FUTURE-scene"
#     threedfuture_input_directory_images = os.path.join(threedfuture_main_source_folder, 'train', f"image_512_{key}_f")
#     threedfuture_input_directory_captions = os.path.join(threedfuture_main_source_folder, 'train', f"image_512_{key}_f_captions")

#     shutil.copy2(threedfuture_input_directory_images, output_directory_images)
#     shutil.copy2(threedfuture_input_directory_captions, output_directory_captions)

#     # IKEA Data
#     ikea_main_source_folder = "./ikea_room_scenes"
#     ikea_input_directory = os.path.join(main_source_folder, 'room_scenes_512_' + key)
#     ikea_input_directory_images = os.path.join(ikea_input_directory, "images")
#     ikea_input_directory_captions = os.path.join(ikea_input_directory, "captions")

#     shutil.copy2(ikea_input_directory_images, output_directory_images)
#     shutil.copy2(ikea_input_directory_captions, output_directory_captions)

In [90]:
combine_data("living")

PermissionError: [Errno 13] Permission denied: './3D-FUTURE-scene\\train\\image_512_living_f'